    11## POS tagging using modified Viterbi

### 1. Data Preparation

In [1]:
#Importing libraries
import numpy as np
import pandas as pd

import nltk

import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

#### Exploring corpus

In [3]:
# checking tagged sentences
nltk_data[:2]

[[('Pierre', 'NOUN'),
  ('Vinken', 'NOUN'),
  (',', '.'),
  ('61', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  (',', '.'),
  ('will', 'VERB'),
  ('join', 'VERB'),
  ('the', 'DET'),
  ('board', 'NOUN'),
  ('as', 'ADP'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
  ('director', 'NOUN'),
  ('Nov.', 'NOUN'),
  ('29', 'NUM'),
  ('.', '.')],
 [('Mr.', 'NOUN'),
  ('Vinken', 'NOUN'),
  ('is', 'VERB'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Elsevier', 'NOUN'),
  ('N.V.', 'NOUN'),
  (',', '.'),
  ('the', 'DET'),
  ('Dutch', 'NOUN'),
  ('publishing', 'VERB'),
  ('group', 'NOUN'),
  ('.', '.')]]

In [4]:
# unique tags applied to the words in the corpus
print(set([tpl[1] for pair in nltk_data for tpl in pair]))

{'ADP', 'ADJ', 'PRON', 'VERB', 'NOUN', '.', 'CONJ', 'NUM', 'PRT', 'X', 'DET', 'ADV'}


In [5]:
# checking unique words in the corpus, which does not have 'X', 'NUM', or '.' tags.

print(len(set([tlp[0] for pair in nltk_data for tlp in pair 
                                                  if (tlp[1] != 'X' and tlp[1] != '.' and tlp[1] != 'NUM')
              ]
         )))

10992


In [6]:
# checking the text for which 'NOUN' tag is used

print(set(
    [tpl[0].lower() for pair in nltk_data for tpl in pair 
                                        if tpl[1] == 'NOUN']
         ))

{'telerate', 'office', 'agriculture', 'chrysler', 'george', 'operators', 'cs', 'seafood', 'signore', 'look', 'three-sevenths', 'xerox', 'roederer', 'leinonen', 'li', 'guards', 'kidder', 'approval', 'slides', 'haden', 'campeau', 'clerks', 'caldor', 'carballo', 'kappa', 'kong', 'knowns', 'dahl', 'hollis', 'investors', 'envoy', 'valhi', 'sidewalk', 'poverty', 'commonwealth', 'gains', 'departments', 'taipei', 'comprehensive', 'railings', 'economics', 'clara', 'american', 'rating', 'composting', 'drives', 'form', 'necessities', 'lady', 'escape', 'peng', 'wafa', 'maytag', 'nelms', 'veteran', 'huxtable', 'series', 'crisis', 'intraday', 'cannell', 'shipyards', 'child', 'californian', 'philip', 'ferro', 'bernstein', 'biaggi', 'philadelphia', 'children', 'corp', 'attwood', 'cents', 'blancs', 'counterweight', 'f-series', 'buckhead', 'machinery', 'powder', '1\\/10th', 'pierre', 'virginians', 'name', 'nrdc', 'hopwood', 'blank', 'americans', 'copper', 'murray', 'platitudes', 'equipment', 'squier', '

In [7]:
# checking the text for which 'X' tag is used

print(set(
    [tpl[0].lower() for pair in nltk_data for tpl in pair 
                                        if tpl[1] == 'X']
         ))

# as per the documentation 'X' tag is used for foreign words.
# https://www.nltk.org/_modules/nltk/tag/mapping.html

{'*t*-64', '*-153', '*-41', '*t*-179', '*t*-39', '*-88', '*t*-90', '*t*-224', '*-102', '*not*', '*-165', '*t*-219', '*ich*-3', '*-18', '*-49', '*t*-68', '*t*-85', '*t*-164', '*-111', '*t*-3', '*exp*-2', '*-80', '*t*-109', '*t*-99', '*t*-10', '*t*-5', '*t*-111', 'a', '*t*-212', '*t*-149', '*-14', 'etc.', '*-133', '*t*-223', 'ok', '*t*-112', '*t*-43', '*t*-229', '*t*-185', '*-140', '*-74', '*t*-35', '*t*-31', '*-67', '*t*-135', '*-162', '*t*-240', '*t*-2', '*t*-235', '*t*-61', '*?*', '*t*-248', '*t*-25', '*t*-156', '*-96', '*t*-70', '*t*-9', '*t*-117', '*t*-151', '*t*-115', '*-21', '*t*-124', '*t*-72', '*t*-26', '*t*-1', '*t*-118', '*-105', '*t*-203', '*t*-24', '*t*-215', '*exp*-3', '*-13', '*t*-27', '*t*-22', '*-31', '*t*-228', '*t*-84', '*-40', '*t*-253', '*t*-69', '*-129', '*-16', '*t*-29', '*t*-130', '*-141', '*ppa*-3', '*u*', '*-59', '*-64', '*t*-110', '*t*-231', '*-19', '*t*-82', '*t*-232', '*-1', '*t*-86', '*t*-129', '*t*-49', '*-104', '*-124', '*-99', '*-76', '*t*-173', '*t*-126'

#### Splitting the data into train and test sets

In [8]:
# as per the instructions, splitting the data into 95% train set and 5% test set
random.seed(1234)
train_set, test_set = train_test_split(nltk_data,test_size=0.05)

print(len(train_set))
print(len(test_set))
print(train_set[:5])

3718
196
[[('Reliance', 'NOUN'), ('acquired', 'VERB'), ('a', 'DET'), ('7', 'NUM'), ('%', 'NOUN'), ('UAL', 'NOUN'), ('stake', 'NOUN'), ('early', 'ADV'), ('this', 'DET'), ('year', 'NOUN'), ('at', 'ADP'), ('an', 'DET'), ('average', 'ADJ'), ('cost', 'NOUN'), ('of', 'ADP'), ('$', '.'), ('110', 'NUM'), ('*U*', 'X'), ('a', 'DET'), ('share', 'NOUN'), (',', '.'), ('and', 'CONJ'), ('reduced', 'VERB'), ('its', 'PRON'), ('stake', 'NOUN'), ('to', 'PRT'), ('4.7', 'NUM'), ('%', 'NOUN'), ('after', 'ADP'), ('UAL', 'NOUN'), ('accepted', 'VERB'), ('the', 'DET'), ('bid', 'NOUN'), ('at', 'ADP'), ('prices', 'NOUN'), ('higher', 'ADJ'), ('than', 'ADP'), ('$', '.'), ('282', 'NUM'), ('*U*', 'X'), ('a', 'DET'), ('share', 'NOUN'), ('.', '.')], [('The', 'DET'), ('issue', 'NOUN'), ('is', 'VERB'), ('further', 'ADV'), ('complicated', 'ADJ'), ('*-1', 'X'), ('by', 'ADP'), ('uncertainty', 'NOUN'), ('over', 'ADP'), ('the', 'DET'), ('future', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('U.S.', 'NOUN'), ("'s", 'PRT'), ('lease

### 2. Build the vanilla Viterbi based POS tagger

In [9]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]

# number of tags
T = set([pair[1] for pair in train_tagged_words])

#### Emission probability

In [10]:
# compute word given tag: Emission Probability

def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1].lower()==tag.lower()]
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0].lower()==word.lower()]
    
    return (len(w_given_tag_list), len(tag_list))

#### Transition probability

In [11]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

#### Viterbi Algorithm

In [12]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)):
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [13]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

In [14]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    
    #extracting unique tags from the text
    T = list(set([pair[1] for pair in train_bag]))
    
    #enumerate creates a tuple with key and the individual value passed in the enumerate function.
    #in this case words. 
    
    #E.g. l1 = ["eat","sleep","repeat"] 
    # print(l1) will result in (0, 'eat'), (1, 'sleep'), (2, 'repeat')
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            #######
            #print("%s %s %s" % (emission_p, word, tag))
            #######
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
    # the loop executes for (words*tags) times. Therefore time is O(nt); where, n=words and t=tags.
            
        #print(p)
        pmax = max(p)
        ######
        #print('pmax = %f, %s, %s' % (pmax, word, tag))
        ######
        
        
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        ######
        #print('pmax = %f, %s, %s' % (pmax, word, state_max))
        ######
        
        state.append(state_max)
    return list(zip(words, state))

In [15]:
# Viterbi Heuristic
def Viterbi_updated(words, train_bag = train_tagged_words):
    
    #########################################################################
    # Defining RegEx and Unigram tagger for tagging unknown words
    
    # specifying patterns for tagging as per the patterns found in the test sentences
    # example from the NLTK book
    patterns = [
        (r'.*ing$', 'VERB'),                             # gerund
        (r'.*ed$', 'VERB'),                              # past tense
        (r'.*es$', 'VERB'),                              # 3rd singular present
        (r'.*ould$', 'PRT'),                             # modals
        (r'[A-Z]{1}([a-z]{1,})?([A-Z]{1,})?', 'NOUN'),   # words with capital letters
        (r'.*\'s$', 'NOUN'),                             # possessive nouns
        (r'.*s$', 'NOUN'),                               # plural nouns
        (r'([0-9])+(\-|\/)?([a-z]{2,3})?(\-)?', 'NUM'),  # cardinal numbers and dates
        (r'.*', 'NOUN')                                  # any word not found will be tagged as a noun
    ]
    
    
    
    rule_based_tagger = nltk.RegexpTagger(patterns)
    
    # lexicon tagger is trained on the NLTK's Universal corpus.
    # lexicon tagger, backed up by the rule-based tagger
    lexicon_tagger = nltk.UnigramTagger(train_set, backoff=rule_based_tagger)
    #########################################################################
    
    state = []
    
    #extracting unique tags from the text
    T = list(set([pair[1] for pair in train_bag]))
    
    #enumerate creates a tuple with key and the individual value passed in the enumerate function.
    #in this case words. 
    
    #E.g. l1 = ["eat","sleep","repeat"] 
    # print(l1) will result in (0, 'eat'), (1, 'sleep'), (2, 'repeat')
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            #######
            #print("%s %s %s" % (emission_p, word, tag))
            #######
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
    # the loop executes for (words*tags) times. Therefore time is O(nt); where, n=words and t=tags.
            
        #print(p)
        pmax = max(p)
        ######
        #print('pmax = %f, %s, %s' % (pmax, word, tag))
        ######
        
        if pmax == 0.0:
            # if the emission probability of a word is 0 (meaning it's an unknown word) then use unigram tagger
            state_max = (lexicon_tagger.tag([word])[0])[1]
            
        else:
            # getting state for which probability is maximum
            state_max = T[p.index(pmax)] 

        
        state.append(state_max)
    return list(zip(words, state))

#### Evaluation on Test data set

In [16]:
# creating a function to show list of words which were incorrectly tagged.

def correction(output_of_viterbi_algo, testing_set):
    return [
                [
                    j
                ] 

                for i, j in enumerate(zip(output_of_viterbi_algo, testing_set))
                    if j[0]!=j[1]
            ]

### 3. Solve the problem of unknown words

#### 3.1 Analyzing how Viterbi Algoritm tags the new words

Identifying patterns which may help in improving the problem of unknown words.

In [18]:
# reading the Test_sentences.txt file and applying the viterbi algo, to see how the words are tagged.

test_text = open('Test_sentences.txt', 'r')
words = word_tokenize(test_text.read())
viterbi_tagged_wrds = Viterbi(words)
print('Vanilla Viterbi completed')

#viterbi_tagged_wrds_upd = Viterbi_updated(words)
print('Updated Viterbi completed')

Vanilla Viterbi completed
Updated Viterbi completed


In [20]:
viterbi_tagged_wrds

[('Android', 'ADP'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'ADP'),
 ('.', '.'),
 ('Android', 'ADP'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'ADP'),
 ('worldwide', 'ADP'),
 ('on', 'ADP'),
 ('smartphones', 'ADP'),
 ('since', 'ADP'),
 ('2011', 'ADP'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'ADP'),
 ('.', '.'),
 ('Google', 'ADP'),
 ('and', 'CONJ'),
 ('Twitter', 'ADP'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'ADP'),
 ('that', 'DET'),
 ('gave', 'VERB'),
 ('Google', 'ADP'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'ADP'),
 ("'s", 'PRT'),
 ('firehose', 'ADP'),
 ('.', '.'),
 ('Twitter', 'ADP'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'ADP'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('service', 'NOU

Here we can see that words (nouns) like 'Android', 'Google', 'Twitter', 'FIFA' etc. are tagged as 'ADP - adpositions (prepositions and postpositions)', because they were not present in the earlier corpus.

### 4. Evaluating tagging accuracy

#### Evaluating tagging accuracy on Viterbi algorithm (checking Universal's train data vs test data)

In [95]:
# Running on entire test dataset would take more than 3-4hrs. 
# Let's test our Viterbi algorithm on a few sample sentences of test dataset

random.seed(1234)

# choose random 20 sents
rndom = [random.randint(1,len(test_set)) for x in range(20)]

# list of sents
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [96]:
# tagging the test sentences
tagged_seq = Viterbi(test_tagged_words)

In [97]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]
print(len(check)/len(tagged_seq))

0.9272388059701493


#### Evaluating tagging accuracy of Lexicon and Rule based taggers

In [98]:
# Rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

# lexicon tagger is trained on the NLTK's Universal corpus.
lexicon_tagger = nltk.UnigramTagger(train_set)

In [99]:
lexicon_tagger.evaluate(test_set)

0.9055520102105935

In [100]:
rule_based_tagger.evaluate(test_set)

0.33843863007870667

### 5. List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [32]:
correction(viterbi_tagged_wrds, viterbi_tagged_wrds_upd)

[[(('Android', 'DET'), ('Android', 'NOUN'))],
 [(('Google', 'DET'), ('Google', 'NOUN'))],
 [(('Android', 'DET'), ('Android', 'NOUN'))],
 [(('OS', 'DET'), ('OS', 'NOUN'))],
 [(('worldwide', 'DET'), ('worldwide', 'NOUN'))],
 [(('smartphones', 'DET'), ('smartphones', 'VERB'))],
 [(('2011', 'DET'), ('2011', 'NUM'))],
 [(('2013', 'DET'), ('2013', 'NUM'))],
 [(('Google', 'DET'), ('Google', 'NOUN'))],
 [(('Twitter', 'DET'), ('Twitter', 'NOUN'))],
 [(('2015', 'DET'), ('2015', 'NUM'))],
 [(('Google', 'DET'), ('Google', 'NOUN'))],
 [(('Twitter', 'DET'), ('Twitter', 'NOUN'))],
 [(("'s", 'VERB'), ("'s", 'PRT'))],
 [(('firehose', 'DET'), ('firehose', 'NOUN'))],
 [(('Twitter', 'DET'), ('Twitter', 'NOUN'))],
 [(('online', 'DET'), ('online', 'NOUN'))],
 [(('networking', 'DET'), ('networking', 'VERB'))],
 [(('interact', 'DET'), ('interact', 'NOUN'))],
 [(('messages', 'DET'), ('messages', 'VERB'))],
 [(('known', 'ADJ'), ('known', 'VERB'))],
 [(('tweets', 'DET'), ('tweets', 'NOUN'))],
 [(('domineering', 